In [39]:
from bs4 import BeautifulSoup
import urllib.request
from datetime import date
import pandas as pd
date_today = date.today()

product_url = []
picture_url = []
brand = []
product_type = []
timestamp = []
name = []
    
items = ['jackets', 'shirts', 'tops_sweaters', 'sweatshirts', 'pants']
for item in items:
    url = 'https://www.supremenewyork.com/shop/all/{}'.format(item)
    ourUrl = urllib.request.urlopen(url)
    soup = BeautifulSoup(ourUrl,'html.parser')
    
    for i in soup.find('ul',{'class':'turbolink_scroller'}).find_all('li'):
    
        timestamp.append(date_today)
        brand.append('Supreme')
        product_type.append(item) # change
    
        product_url.append("https://www.supremenewyork.com" + i.find('div',{'class':'inner-article'}).a.get('href'))
        picture_url.append("https:" + i.find('div',{'class':'inner-article'}).img.get('src').replace('vi','ma'))
 
        name.append(i.find('a',{'class':'name-link'}).text)

In [33]:
color = []
product_description = []
material = []
picture_name = []
price = []
sales_price = []

counter = 1

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')

driver = webdriver.Chrome(options=options, executable_path=r'/Users/mandili/.wdm/drivers/chromedriver/87.0.4280.20/mac64/chromedriver')

for url in product_url:
    
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.execute_script("return document.body.innerHTML;"),'html.parser')
    
    picture_name.append('supreme_' + str(counter) + '.jpg')
    counter += 1
    item_color = []
    for item_i in soup.find('div',{'id':'details'}).ul.find_all('li'):
        item_color.append(item_i.button.get('data-style-name'))
    color.append(item_color)
    
    sales_price.append(soup.find('p',{'class':'price'}).text)
    price.append(None)
    
    product_description.append(soup.find('p',{'itemprop':'description'}).text)
    material.append(None)

In [40]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", 'picture_name']
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = name
second_pass.product_description = product_description
second_pass.price = price
second_pass.sales_price = sales_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = picture_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass.head()

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,Supreme,jackets,Supreme®/The North Face® Faux Fur Nuptse Jacket,Faux fur with 700-Fill down insulation and Win...,None,$578,"[Green, Red, Black]",None,https://www.supremenewyork.com/shop/jackets/x1...,https://assets.supremenewyork.com/199986/ma/8Q...,2020-12-12,supreme_1.jpg
1,Supreme,jackets,Supreme®/The North Face® Faux Fur Nuptse Jacket,Faux fur with 700-Fill down insulation and Win...,None,$578,"[Green, Red, Black]",None,https://www.supremenewyork.com/shop/jackets/x1...,https://assets.supremenewyork.com/199981/ma/Ef...,2020-12-12,supreme_2.jpg
2,Supreme,jackets,Supreme®/The North Face® Faux Fur Nuptse Jacket,Faux fur with 700-Fill down insulation and Win...,None,$578,"[Green, Red, Black]",None,https://www.supremenewyork.com/shop/jackets/x1...,https://assets.supremenewyork.com/199976/ma/3E...,2020-12-12,supreme_3.jpg
3,Supreme,jackets,Script Logos Wool Overcoat,Wool blend with jacquard logo pattern and visc...,None,$558,"[Tan, Black]",None,https://www.supremenewyork.com/shop/jackets/bj...,https://assets.supremenewyork.com/193561/ma/qK...,2020-12-12,supreme_4.jpg
4,Supreme,jackets,Script Logos Wool Overcoat,Wool blend with jacquard logo pattern and visc...,None,$558,"[Tan, Black]",None,https://www.supremenewyork.com/shop/jackets/bj...,https://assets.supremenewyork.com/198943/ma/PQ...,2020-12-12,supreme_5.jpg


In [41]:
second_pass.to_csv('supreme.csv', index = False, encoding='utf-8-sig')

In [38]:

import os
import requests as req

new_pd = pd.read_csv('/Users/mandili/Desktop/supreme.csv', encoding='utf-8-sig')

os.mkdir('supreme')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('supreme/'+ "supreme_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1